<a href="https://colab.research.google.com/github/swarthyPig/Raspberry-Pi_DHT22_Weather_Station_with_correlation-and-ML/blob/master/machine%20learning/Multi_Variable_TF_MLR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.parsers import read_csv

model = tf.global_variables_initializer();

 
data = read_csv('iot_data.csv', sep=',')

# date는 data type이 float이 아니므로 오류발생해서 삭제했습니다.
# csv 파일안의 행 순서는 index_number, temp, humi, cr 입니다.

xy = np.array(data, dtype=np.float32)

x_data = xy[0:-50, 1:-1] # temp, humi

y_data = xy[0:-50, [-1]] # cr

x_data_test = xy[-50:, 1:-1] # test data temp, humi

y_data_test = xy[-50:, [-1]] # test data cr


# 행렬의 Shape을 None으로 지정하는 것은 N개의 데이터를 받겠다는 의미이다.(N x 2 행렬)

X = tf.placeholder(tf.float32, shape=[None, 2])

Y = tf.placeholder(tf.float32, shape=[None, 1])


# 각 변수를 array가 아닌 matrix로 나타냅니다.

W = tf.Variable(tf.random_normal([2, 1]), name="weight")

b = tf.Variable(tf.random_normal([1]), name="bias")


# matrix로 나타낸 변수를 행렬곱을 사용하여 가설을 세웁니다.

hypothesis = tf.matmul(X, W) + b


# 비용 함수를 설정합니다.

cost = tf.reduce_mean(tf.square(hypothesis - Y))


# 경사하강 라이브러리
# 최적화 함수를 설정합니다.

optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.0001)

train = optimizer.minimize(cost)



# 세션을 생성합니다.

sess = tf.Session()



# 글로벌 변수를 초기화합니다.

sess.run(tf.global_variables_initializer())



# 학습을 수행합니다.

for step in range(100001):

    cost_, hypo_, _ = sess.run([cost, hypothesis, train], feed_dict={X: x_data, Y: y_data})

    if step % 10000 == 0:

        print("#", step, " 손실 비용: ", cost_)
        print("- Weigth : ", sess.run(W[0]),sess.run(W[1]))
        print("- Bias : ", sess.run(b))
        print("- 쾌적도: ", hypo_[0])
        

saver = tf.train.Saver()

save_path = saver.save(sess, "saved.cpkt")

print('학습된 모델을 저장했습니다.')




Instructions for updating:
Colocations handled automatically by placer.
# 0  손실 비용:  670.20294
- Weigth :  [0.03520169] [-0.18859364]
- Bias :  [-1.1970011]
- 쾌적도:  [-24.932251]
# 10000  손실 비용:  1.1048441
- Weigth :  [0.03480503] [0.09850135]
- Bias :  [-1.2039334]
- 쾌적도:  [4.6826644]
# 20000  손실 비용:  1.104754
- Weigth :  [0.03503799] [0.09858751]
- Bias :  [-1.2134701]
- 쾌적도:  [4.682611]
# 30000  손실 비용:  1.1046641
- Weigth :  [0.03527099] [0.09867366]
- Bias :  [-1.2230068]
- 쾌적도:  [4.6825576]
# 40000  손실 비용:  1.1045747
- Weigth :  [0.03550395] [0.09875983]
- Bias :  [-1.2325436]
- 쾌적도:  [4.6825037]
# 50000  손실 비용:  1.1044854
- Weigth :  [0.03573691] [0.09884599]
- Bias :  [-1.2420803]
- 쾌적도:  [4.6824503]
# 60000  손실 비용:  1.1043966
- Weigth :  [0.03596989] [0.09893215]
- Bias :  [-1.2516171]
- 쾌적도:  [4.682397]
# 70000  손실 비용:  1.104308
- Weigth :  [0.03620286] [0.09901831]
- Bias :  [-1.2611538]
- 쾌적도:  [4.6823435]
# 80000  손실 비용:  1.1042199
- Weigth :  [0.03643583] [0.09910448]
- Bia

In [2]:
# 온,습도 변수의 값을 입력 받습니다.

temperature = float(input('온도: '))

humidity = float(input('습도: '))


with tf.Session() as sess:

    sess.run(model)


    # 저장된 학습 모델을 파일로부터 불러옵니다.
    
    save_path = "saved.cpkt"
    
    saver.restore(sess, save_path)



    # 사용자의 입력 값을 이용해 배열을 만듭니다.
    
    data = ((temperature, humidity), )
    
    arr = np.array(data, dtype=np.float32)



    # 예측을 수행한 뒤에 그 결과를 출력합니다.
    
    x_data = arr[0:2]
    
    dict = sess.run(hypothesis, feed_dict={X: x_data})

    print(dict[0])
    
    # Show the values of weights and bias
    print('Weights:')
    print(sess.run(W))
    print('Bias:')
    print(sess.run(b))

온도: 23
습도: 43
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from saved.cpkt
[3.827879]
Weights:
[[0.03690177]
 [0.0992768 ]]
Bias:
[-1.289764]
